In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import json
import os
import plotly.express as px
from collections import Counter
import re
import numpy as np

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
import category_encoders as ce

In [3]:
from sklearn import set_config
set_config(display='diagram')

# Files.

The kit is composed of three files:
- `vins_train.csv`, containing a list of wines with features,
- `producteurs.geojson`, containing the list of producers with thair address and geolocation,

# Data presentation.

In [4]:
df_producteurs = pd.read_csv('producteurs.csv', sep=';')

In [5]:
df_vins_train = pd.read_csv('vins_train.csv', sep=';')

In [6]:
df_vins = df_vins_train.merge(df_producteurs.drop(columns=['producteur']), how='left', on='prod_id')

In [11]:
fig = px.scatter_mapbox(df_vins,
                        lat='lat', lon='lon', 
                        hover_name="region",
                        color='region',
                        hover_data=['producteur'],
                        zoom=5)
fig.update_layout(title="Carte de France des producteurs de vins", mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
fig.write_html('Carte de France - Producteurs.html', auto_open=True)

In [12]:
fig = px.scatter_mapbox(df_vins,
                        lat='lat', lon='lon', 
                        hover_name="vin",
                        color='appellation',
                        hover_data=['producteur', 'region'],
                        zoom=5)
fig.update_layout(title="Carte de France des vins", mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
fig.write_html('Carte de France - Appellations.html', auto_open=True)

In [13]:
df_vins_train.head()

,vin,cuvee,millesime,guide,note,classement,region,appellation,type,prix,prix_min,prix_max,prix_m,consommation,commentaire,garde,production,elevage,producteur,prod_id
0,CHÂTEAU MOUSSEYRON,NaN,2008.0,Guide 2010,1.0,NaN,Bordelais,Bordeaux blanc,Blanc tranquille,- de 5 €,NaN,5.0,5.0,NaN,Vin très réussi,NaN,15000,En cuve,SCEA JORIS LARRIAUT,13733.0
1,CHÂTEAU HAUT CORBIAN,NaN,2014.0,Guide 2018,1.0,NaN,Bordelais,Saint-estèphe,Rouge tranquille,11 à 15 €,11.0,15.0,13.0,A boire,Vin très réussi,2020 - 2024,6000,En fût,HAUT COTEAU,48920.0
2,DOMAINE BELLE,NaN,2010.0,Guide 2014,0.0,NaN,Vallée du Rhône,Hermitage,Rouge tranquille,30 à 50 €,30.0,50.0,40.0,NaN,Vin cité,NaN,3500,En fût,EARL LES MARSURIAUX,49569.0
3,COUVREUR-PHILIPPART,Carte d'or,NaN,Guide 2016,1.0,1er cru,Champagne,Champagne,Blanc effervescent,11 à 15 €,11.0,15.0,13.0,A boire,Vin très réussi,2015 - 2017,30000,En cuve,EMMANUEL COUVREUR,67163.0
4,DOMAINE MADELOC,Serrail,2011.0,Guide 2015,2.0,NaN,Roussillon,Collioure,Rouge tranquille,11 à 15 €,11.0,15.0,13.0,A boire,Vin remarquable,2014 - 2018,20000,En cuve et en fût,SARL PIERRE GAILLARD (DOM. MADELOC),48072.0


In [15]:
df_producteurs.head()

,producteur,adresse,CP,commune,pays,INSEE_COM,INSEE_DEP,INSEE_REG,prod_id,lon,lat
0,Cigalart,"rue Bonnechère, 39",4367,Crisnée,Belgique,NaN,NaN,NaN,67152,NaN,NaN
1,Hesby-Vins,"rue Bonnechère, 39",4367,Crisnée,Belgique,NaN,NaN,NaN,50117,NaN,NaN
2,Jean-christophe Pellerin,Perrozan,1150,Vaux-en-Bugey,France,01431,01,84.0,50416,5.353699,45.927471
3,Marcel Perinet,"65, chem. du Péroud",1150,Saint-Martin-du-Mont,France,01374,01,84.0,19862,5.327717,46.101552
4,Thierry Tissot,"42, quai du Buizin",1150,Vaux-en-Bugey,France,01431,01,84.0,35256,5.353699,45.927471


Wine's features:
- 'vin': wine name,
- 'cuvee'
- 'millesime': wine vintage,
- 'guide': Hachette wine guide vintage of the evaluation,
- 'note' : wine score according Hachette wine guide (from 0 to 3). Unknown scores are also noted as 0.
- 'classement': wine classification,
- 'region' 
- 'appellation',
- 'type',
- 'consommation': comment concerning wine consumption,
- 'commentaire': comment one wine's quality,
- 'garde': when filled, conservation duration,
- 'production': the number of bottles produced for this wine,
- 'elevage': the way the wine grew (barrel, tank...)
- 'producteur': producer's name
- 'prod_id': producer's identification number

Producer's features:
- 'producteur': producer's name,
- 'adresse': producer's address,
- 'CP': postal zip code,
- 'commune': city,
- 'pays': country,
- 'INSEE_COM': administrative zip code,
- 'INSEE_DEP': department administrative zip code,
- 'INSEE_REG': region administrative zip code,
- 'prod_id': producer's identification number,
- 'lon': geographic coordinate - longitude
- 'la': geographic coordinate - latitude

# Basic model.

In [12]:
df_producteurs = pd.read_csv('producteurs.csv', sep=';')
df_producteurs

,producteur,adresse,CP,commune,pays,INSEE_COM,INSEE_DEP,INSEE_REG,prod_id,lon,lat
0,Cigalart,"rue Bonnechère, 39",4367,Crisnée,Belgique,NaN,NaN,NaN,67152,NaN,NaN
1,Hesby-Vins,"rue Bonnechère, 39",4367,Crisnée,Belgique,NaN,NaN,NaN,50117,NaN,NaN
2,Jean-christophe Pellerin,Perrozan,1150,Vaux-en-Bugey,France,01431,01,84.0,50416,5.353699,5.353699
3,Marcel Perinet,"65, chem. du Péroud",1150,Saint-Martin-du-Mont,France,01374,01,84.0,19862,5.327717,5.327717
4,Thierry Tissot,"42, quai du Buizin",1150,Vaux-en-Bugey,France,01431,01,84.0,35256,5.353699,5.353699
...,...,...,...,...,...,...,...,...,...,...,...
17070,Gebrüder Kümin,"Weinbau & Weinnandel AG, Oechsli 1",8807,Freienbach,Suisse,NaN,NaN,NaN,34379,NaN,NaN
17071,Alois Walser & Co,Hofreiti,8885,Mols,Suisse,NaN,NaN,NaN,46989,NaN,NaN
17072,Emil Nüesch AG,Hauptstr. 71,9436,Balgach,Suisse,NaN,NaN,NaN,46987,NaN,NaN
17073,Jakob Schmid AG,Tramstrasse 23,9442,Berneck,Suisse,NaN,NaN,NaN,17363,NaN,NaN


## Train set preparation.

In [13]:
df_train = pd.read_csv('vins_train.csv', sep=';')
df_train = df_train.merge(df_producteurs.drop(columns=['producteur']), how='left', on='prod_id')
# Elimination des vins dont on ne dispose pas du millesime ou du prix
df_train = df_train[~df_train['millesime'].isna()].reset_index(drop=True)
df_train = df_train[~df_train['prix_m'].isna()].reset_index(drop=True)

df_Y_train = df_train[['prix_m']].copy()

# Elimination de certaines colonnes
df_X_train = df_train.drop(columns=['guide', 'prix', 
                                    'prix_min', 'prix_max', 'prix_m',  
                                    'garde', 'cuvee', 
                                    'prod_id', 
                                    'adresse', 'CP', 'commune', 'pays', 
                                    'INSEE_COM', 'INSEE_DEP', 'INSEE_REG'])

In [14]:
df_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105432 entries, 0 to 105431
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   vin           105432 non-null  object 
 1   millesime     105432 non-null  float64
 2   note          105432 non-null  float64
 3   classement    10815 non-null   object 
 4   region        105432 non-null  object 
 5   appellation   105432 non-null  object 
 6   type          105432 non-null  object 
 7   consommation  32602 non-null   object 
 8   commentaire   101963 non-null  object 
 9   production    105432 non-null  int64  
 10  elevage       105432 non-null  object 
 11  producteur    104823 non-null  object 
 12  lon           102843 non-null  float64
 13  lat           102843 non-null  float64
dtypes: float64(4), int64(1), object(9)
memory usage: 11.3+ MB


## Test set preparation.

In [15]:
df_test = pd.read_csv('vins_test.csv', sep=';')
df_test = df_test.merge(df_producteurs.drop(columns=['producteur']), how='left', on='prod_id')

# Elimination des vins dont on ne dispose pas du millesime ou du prix
df_test = df_test[~df_test['millesime'].isna()].reset_index(drop=True)
df_test = df_test[~df_test['prix_m'].isna()].reset_index(drop=True)

df_Y_test = df_test[['prix_m']].copy()

# Elimination de certaines colonnes
df_X_test = df_test.drop(columns=['guide', 'prix',
                                  'prix_min', 'prix_max', 'prix_m',
                                  'garde', 'cuvee',
                                  'prod_id',
                                  'adresse', 'CP', 'commune', 'pays',
                                  'INSEE_COM', 'INSEE_DEP', 'INSEE_REG'])

## Model definition.

In [16]:
ohe_cols = ['classement', 'region', 'commentaire', 'type', 'consommation', 'elevage']
tgt_cols = ['vin', 'producteur', 'appellation']

In [17]:
ohe_pipeline = Pipeline([
    #("imputer", SimpleImputer(strategy='constant', fill_value='missing')),
    ("one-hot-encoder", OneHotEncoder(handle_unknown='ignore'))
])

tgt_pipeline = Pipeline([
    ("target-encoder", ce.TargetEncoder())
])

preprocessing = ColumnTransformer([
    ("ohe_preproc", ohe_pipeline, ohe_cols),
    ("tgt_preproc", tgt_pipeline, tgt_cols)
])

model = Pipeline([
    ("Preprocessing", preprocessing),
    ("regressor", LinearRegression())
])
model.fit(df_X_train, df_Y_train)

C:\Users\ROG-Hub\anaconda3\envs\Travail\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('ohe_preproc',
                                                  Pipeline(steps=[('one-hot-encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['classement', 'region',
                                                   'commentaire', 'type',
                                                   'consommation', 'elevage']),
                                                 ('tgt_preproc',
                                                  Pipeline(steps=[('target-encoder',
                                                                   TargetEncoder())]),
                                                  ['vin', 'producteur',
                                                   'appellation'])])),
                ('regressor', LinearRegression())])

In [18]:
model.score(df_X_train, df_Y_train)

0.38364817076145696

In [19]:
model.score(df_X_test, df_Y_test)

0.4993277362845129